# Pre-process Sleep Data

This script loads and pre-processes the sleep polysomnography biosignals from the Cleveland Family Study (https://sleepdata.org/datasets/cfs). We then filter the EEG, EMG, EOG, and ECG signals, re-reference the EEG data to the linked mastoids, and then extract our epochs as an MNE object. The epochs of interest are 4 seconds in length and do not overlap. We then optionally downsample everything to 128 Hz and save the epochs as ".fif.gz" files and the hypnograms as ".npy" files.

In [1]:
%matplotlib inline

## Import packages 
import numpy as np
import yasa
import os
import mne
from tqdm import tqdm
import random
import neurokit2 as nk
import pandas as pd
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
mne.set_log_level('WARNING')

In [49]:
## Additonal useful functions

def process_raw_EDF_cfs(file):
    """
    Process a raw EDF file, apply various preprocessing steps, and return the processed raw data, epochs, and hypnogram.

    This function:
    - Imports the raw EDF file.
    - Maps and picks channels of interest.
    - Rereferences EEG data.
    - Bipolarizes EOG and EMG data.
    - Applies various filters.
    - Epochs the data into fixed-length 4-second segments.
    - Downsamples the data.
    - Imports and unravels the hypnogram.

    Parameters:
    - file (str): Path to the EDF file (without the '.edf' extension).

    Returns:
    - tuple: A tuple containing:
        - raw_train (mne.io.Raw): Processed raw data.
        - epochs (mne.Epochs): Epochs created from the raw data.
        - hypnogram (numpy array): An array representing the hypnogram.

    Raises:
    - ValueError: If there's a mismatch between the hypnogram length and the total number of epochs.
    """

    ## Import raw edf file
    raw_train = mne.io.read_raw_edf(file + '.edf', eog = ['LOC','ROC'],
                                    preload = True, verbose = False)

    ## Create dictionary of channels we are interested in  
    mapping = {'C3': 'eeg',
               'C4': 'eeg',
               'M1': 'eeg',
               'M2': 'eeg',
               'LOC': 'eog',
               'ROC': 'eog',
               'EMG2': 'emg',
               'EMG3': 'emg',
               'ECG1': 'ecg'}

    ## Select channels in object and give labels for channel type
    raw_train.pick_channels(ch_names=list(mapping))
    raw_train.set_channel_types(mapping) 

    ## Rereference eeg data to average of mastoids
    raw_train.set_eeg_reference(ref_channels=['M1','M2']) # type: ignore
        
    ## Bipolarize eog and emg data 
    try:
        raw_train = mne.set_bipolar_reference(raw_train, 'EMG2', 'EMG3')
    except:
        if not isinstance(raw_train, mne.io.Raw):
            ref_inst = mne.io.RawArray(raw_train.get_data(), raw_train.info)
            raw_train = mne.set_bipolar_reference(raw_train, 'EMG2', 'EMG3')

    ## Filter data
    raw_train.filter(picks=['eeg','eog'], l_freq=0.5, h_freq=40)
    raw_train.filter(picks='emg', l_freq=10, h_freq=100)
    # Clean the ECG data with neurokit2
    raw_train.apply_function(fun=nk.ecg_clean, picks='ecg', n_jobs=-1, 
                            channel_wise=True, **dict(sampling_rate=raw_train.info['sfreq'], method='neurokit', powerline=60))
    # Notch filter
    raw_train.notch_filter(freqs=[60, 120], method='spectrum_fit') # type: ignore

    ## Create fixed-length 4 second epochs
    events = mne.make_fixed_length_events(raw_train, duration=4)
    epochs = mne.Epochs(raw_train, events, tmin=0, tmax=3.99,
                        baseline=None, detrend=None, preload=True, reject=None)

    ## Downsample data
    #epochs.resample(128)
    #raw_train.resample(128)

    ## import and unravel hypnogram
    stages, stagelens = read_xml(file + '-nsrr.xml')
    hypnogram = unravel_hypnogram(stages, stagelens)
  
    return raw_train, epochs, hypnogram

def read_xml(file):
    """
    Reads an XML annotation file to extract hypnogram information.

    Args:
    - file (str): Path to the XML file.

    Returns:
    - tuple: Two numpy arrays containing stages and their corresponding lengths.
    """
    
    tree = ET.parse(file)
    root = tree.getroot()

    # Use list comprehensions to extract the relevant data
    stages = [int(child[1].text[-1]) for child in root.iter('ScoredEvent') 
              if child[0].text and 'Stages' in child[0].text]
    stagelens = [int(float(child[3].text) / 30) for child in root.iter('ScoredEvent') 
                 if child[0].text and 'Stages' in child[0].text]

    # Convert lists to numpy arrays
    return np.array(stages, dtype=int), np.array(stagelens, dtype=int)

def unravel_hypnogram(stages, stagelens):
    """
    Construct a hypnogram based on provided sleep stages and their durations.

    The function maps each sleep stage to a respective value and then creates
    a continuous array representing the hypnogram.

    Parameters:
    - stages (list or array-like): A list of sleep stages, where each stage is an integer.
    - stagelens (list or array-like): A list of durations (in 30s increments) corresponding to each sleep stage.

    Returns:
    - numpy array: A continuous array representing the hypnogram.

    Raises:
    - ValueError: If the length of the constructed hypnogram does not match the total duration specified by stagelens.
    """

    # Map stages to their respective values
    stage_map = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 3,  # collapse stage 3 and 4
        5: 4
    }

    # Construct the hypnogram using list comprehension and the mapping
    hypnogram = np.concatenate([stage_map[stage] * np.ones(length) for stage, length in zip(stages, stagelens)])
    
    # Sanity check
    if len(hypnogram) != stagelens.sum():
        raise ValueError('The length of the scaled hypnogram does not match the amount of total epochs')
    
    return hypnogram

def downsample_hypnogram(sf, hypno, epochs):
    """
    Adjust the size of a hypnogram to match the number of epochs.

    Parameters:
    -----------
    sf : float or None
        Sampling frequency of the data. If None, duration difference will not be computed.
    hypno : ndarray
        The hypnogram array to be adjusted.
    epochs : mne.Epochs
        The epochs object whose number of epochs the hypnogram should match.

    Returns:
    --------
    ndarray
        Adjusted hypnogram with size matching the number of epochs.
    """
    npts_hyp = hypno.size
    npts_data = epochs._data.shape[0]  # using the first dimension to represent the number of epochs

    if npts_hyp < npts_data:
        # Hypnogram is shorter than data
        hypno = np.pad(hypno, (0, npts_data - npts_hyp), mode="edge")

    elif npts_hyp > npts_data:
        # Hypnogram is longer than data
        hypno = hypno[:npts_data]

    return hypno


## 1. Load data

In [3]:
path = '/media/administrator/data/cfs/polysomnography/'
save_path = '/mnt/server/data03/2023_NENA_Aperiodic_Workshop/data/processed/'
fig_path = '/mnt/server/data03/2023_NENA_Aperiodic_Workshop/figures/subject/'
# Iterate over all files if the names end in .edf
files = [os.path.splitext(f)[0] for f in os.listdir(path) if f.endswith('.edf')]
# Randomly select 50 files 
files = random.sample(files, 50)

## 2. Process and save data

In [4]:
# Iterate over all files and process them
for idx, file in enumerate(tqdm(files)):
    print(f'Preprocessing file : {file}')
    raw, epochs, hypnogram = process_raw_EDF_cfs(path + file)
    # save the data
    epochs.save(fname = save_path + file + '-epo.fif.gz', 
                overwrite=False)
    raw.save(fname = save_path + file + '-raw.fif.gz',  # type: ignore
             overwrite=False)
    np.save(save_path + file + '-hypnogram.npy', hypnogram)
    # delete the data from memory
    del raw, epochs, hypnogram

  0%|          | 0/50 [00:00<?, ?it/s]

Preprocessing file : cfs-visit5-800347


/tmp/ipykernel_507085/142343024.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
  2%|▏         | 1/50 [01:28<1:12:22, 88.62s/it]

Preprocessing file : cfs-visit5-800212


/tmp/ipykernel_507085/142343024.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
  4%|▍         | 2/50 [03:05<1:14:51, 93.57s/it]

...

/tmp/ipykernel_507085/142343024.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
 96%|█████████▌| 48/50 [1:40:49<04:23, 131.70s/it]

Preprocessing file : cfs-visit5-801638


/tmp/ipykernel_507085/142343024.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
 98%|█████████▊| 49/50 [1:43:43<02:24, 144.38s/it]

Preprocessing file : cfs-visit5-801225


/tmp/ipykernel_507085/142343024.py:46: FutureWarning: The default for pick_channels will change from ordered=False to ordered=True in 1.5 and this will result in a change of behavior because the resulting channel order will not match. Either use a channel order that matches your instance or pass ordered=False.
  raw_train.pick_channels(ch_names=list(mapping))
100%|██████████| 50/50 [1:45:26<00:00, 126.52s/it]


# Artifact detection and labeling of Sleep Data 

We then use the pre-processed polysomnography data from above and utilize a Riemanian geometry based algorithm to detect and label artifacts contained therein. The algorithm is based on the following papers: https://hal.archives-ouvertes.fr/hal-00781701 & https://hal.science/hal-02015909 and is implemented in the yasa toolbox. This so-called "Riemannian Potato" is a clustering method that iteratively estimates the centroid of clean signal by rejecting every trial that is too far from it, thus giving you a label for each given 4 second epoch.

## 3. Process & save update epochs and hypnograms

In [50]:
# Obtain list of unique recordings
processed_files = list(set(["-".join(f.split('-')[0:3]) for f in os.listdir(save_path)]))

# Iterate over all files and process them
for idx, file in enumerate(tqdm(processed_files)):
    print(f'Detecting and labeling artifacts in file : {file}')
    # Load the data and hypnogram files
    raw = mne.io.read_raw(save_path + file + '-raw.fif.gz', preload=True) # type: ignore
    epochs = mne.read_epochs(save_path + file + '-epo.fif.gz', preload=True) # type: ignore
    hypnogram = np.load(save_path + file + '-hypnogram.npy')
    # Get sampling frequency
    sf = raw.info['sfreq']
    # Get data
    data = raw.get_data() * 1e6
    # Unravel hypnogram to match data length
    hypnogram_unravel = yasa.hypno_upsample_to_data(hypno=hypnogram, sf_hypno=1/30, 
                                                    data=data, sf_data=sf)
    # Label artifacts based on Riemannian Potato clustering algorithm
    window = 4
    art, zscores = yasa.art_detect(data, sf=sf, window=window, hypno=hypnogram_unravel, 
                                   include=(1, 2, 3, 4), method='covar', threshold=3)
    sf_art = 1 / window
    
    # Upsample art to match data length
    art_up = yasa.hypno_upsample_to_data(art, sf_art, data, sf)

    # Add -1 to hypnogram where artifacts were detected
    hypno_with_art = hypnogram_unravel.copy()
    hypno_with_art[art_up] = -1

    # Plot and save the spectrogram with the updated hypnogram
    fig = yasa.plot_spectrogram(data[1, :], sf, hypno_with_art)
    fig.savefig(fig_path + file + '-hypno-artifacts.png', dpi=300)
    plt.close(fig)

    # Save the updated hypnogram
    np.save(save_path + file + '-hypnogram_with_art.npy', hypno_with_art)

    # Downsample the hypnogram to match the number of epochs
    downsampled_hypnogram = downsample_hypnogram(sf, hypno_with_art, epochs)

    # Create a DataFrame from the downsampled hypnogram
    metadata = pd.DataFrame({'SleepStage': downsampled_hypnogram})
    
    # Add the metadata to the epochs
    epochs.metadata = metadata

    # Save the epochs 
    epochs.save(save_path + file + '-epo.fif.gz', overwrite=True) # type: ignore

    # Delete some objects to free up memory
    del raw, epochs, hypnogram, hypnogram_unravel, art, zscores, art_up, hypno_with_art, downsampled_hypnogram, metadata

  0%|          | 0/50 [00:00<?, ?it/s]

Detecting and labeling artifacts in file : cfs-visit5-801662


25-Aug-23 15:49:29 | WARNING | Hypnogram is SHORTER than data by 2.00 seconds. Padding hypnogram with last value to match data.size.
  2%|▏         | 1/50 [01:40<1:22:13, 100.68s/it]

Detecting and labeling artifacts in file : cfs-visit5-801323


25-Aug-23 15:51:09 | WARNING | Hypnogram is SHORTER than data by 2.00 seconds. Padding hypnogram with last value to match data.size.
  4%|▍         | 2/50 [03:20<1:20:21, 100.46s/it]

...

 96%|█████████▌| 48/50 [2:11:29<04:39, 139.63s/it]

Detecting and labeling artifacts in file : cfs-visit5-801001


25-Aug-23 18:01:01 | WARNING | At least 30 epochs are required to calculate z-score. Skipping stage 1
25-Aug-23 18:01:02 | WARNING | At least 30 epochs are required to calculate z-score. Skipping stage 3
25-Aug-23 18:01:02 | WARNING | At least 30 epochs are required to calculate z-score. Skipping stage 4
25-Aug-23 18:01:02 | WARNING | Hypnogram is SHORTER than data by 2.00 seconds. Padding hypnogram with last value to match data.size.
 98%|█████████▊| 49/50 [2:13:16<02:09, 129.65s/it]

Detecting and labeling artifacts in file : cfs-visit5-802073


25-Aug-23 18:02:55 | WARNING | Hypnogram is SHORTER than data by 2.00 seconds. Padding hypnogram with last value to match data.size.
100%|██████████| 50/50 [2:15:12<00:00, 162.26s/it]


In [ ]:
## Miscellanous
# import pandas as pd
# pd.Series(hypno_with_art).value_counts(normalize=True)
# yasa.sleep_statistics(hypno_with_art, sf_hyp=sf)